In [1]:
# Pricing Options using Fast Fourier Transform (FFT)
# This version uses real-life stock data from Yahoo Finance (yfinance)

# Importing Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# Display settings for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)



In [2]:
# Fetch real-life stock data using yfinance
def fetch_stock_data(ticker, start_date, end_date):
    """
    Fetch stock data using yfinance and calculate additional metrics.
    
    Parameters:
        ticker (str): Stock ticker symbol (e.g., "AAPL" for Apple).
        start_date (str): Start date for the data (YYYY-MM-DD).
        end_date (str): End date for the data (YYYY-MM-DD).

    Returns:
        pd.DataFrame: DataFrame containing stock price and daily return.
    """
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]  # Select relevant columns
    stock_data['Daily Return'] = stock_data['Close'].pct_change()  # Calculate daily returns
    stock_data.dropna(inplace=True)
    return stock_data

# Example: Fetch data for Apple (AAPL)
ticker = "AMZN"
start_date = "2024-01-01"
end_date = "2024-12-31"

stock_data = fetch_stock_data(ticker, start_date, end_date)

# Display the stock data
print(f"Stock data for {ticker}:")
stock_data.tail(10)



[*********************100%***********************]  1 of 1 completed

Stock data for AMZN:


Price,Open,High,Low,Close,Volume,Daily Return
Ticker,AMZN,AMZN,AMZN,AMZN,AMZN,
Date,,,,,,
2024-12-16,230.23,233.00,228.01,232.93,37552100,0.02
2024-12-17,232.39,232.73,227.85,231.15,35948100,-0.01
2024-12-18,230.77,231.40,220.11,220.52,43281400,-0.05
2024-12-19,224.91,226.09,222.92,223.29,39918700,0.01
2024-12-20,219.84,226.21,218.73,224.92,88279200,0.01
2024-12-23,225.01,226.88,223.90,225.06,28070000,0.00
2024-12-24,226.94,229.14,226.13,229.05,15007500,0.02
2024-12-26,228.50,228.50,226.67,227.05,16146700,-0.01


In [3]:
# General Parameters for Option Pricing
S0 = float(stock_data['Close'].iloc[-1])  # Convert to scalar (float)
K = S0 * 1.05                            # Strike price (5% above current stock price)
k = np.log(K)                            # Log strike price
r = 0.03                                 # Risk-free interest rate
q = 0.01                                 # Dividend yield
T = 0.5                                  # Time to maturity (6 months)

# FFT Parameters
n = 12                                   # Exponent for 2^n
N = 2 ** n                               # Number of steps
eta = 0.25                               # Integration step size
alpha = 1.0                              # Damping factor
lambda_ = (2 * np.pi / N) / eta          # Log-strike step size
beta = np.log(K)                         # Log-strike value

# Display Parameters
print(f"Option Pricing Parameters for {ticker}:")
pd.DataFrame({
    'Parameter': ['Initial Stock Price (S0)', 'Strike Price (K)', 'Risk-free Rate (r)', 
                  'Dividend Yield (q)', 'Time to Maturity (T)', 'FFT Steps (N)', 
                  'Integration Step Size (eta)', 'Damping Factor (alpha)'],
    'Value': [S0, K, r, q, T, N, eta, alpha]
})



Option Pricing Parameters for AMZN:


C:\Users\awadh\AppData\Local\Temp\ipykernel_30772\1455031610.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S0 = float(stock_data['Close'].iloc[-1])  # Convert to scalar (float)


,Parameter,Value
0,Initial Stock Price (S0),220.04
1,Strike Price (K),231.04
2,Risk-free Rate (r),0.03
3,Dividend Yield (q),0.01
4,Time to Maturity (T),0.50
5,FFT Steps (N),4096.00
6,Integration Step Size (eta),0.25
7,Damping Factor (alpha),1.00


In [4]:
# Function to Compute Characteristic Function
def compute_characteristic_function(u, params, S0, r, q, T, model):
    """
    Calculate the characteristic function for different option pricing models.

    Parameters:
        u (array): Complex inputs for the Fourier transform.
        params (list): Model-specific parameters.
        S0 (float): Current stock price.
        r (float): Risk-free interest rate.
        q (float): Dividend yield.
        T (float): Time to maturity.
        model (str): Option pricing model ("BS" for Black-Scholes).

    Returns:
        np.ndarray: Values of the characteristic function.
    """
    if model == 'BS':  # Black-Scholes
        volatility = params[0]
        drift = np.log(S0) + (r - q - 0.5 * volatility**2) * T
        diffusion = volatility * np.sqrt(T)
        phi = np.exp(1j * drift * u - 0.5 * (diffusion * u)**2)
    elif model == 'Heston':  # Heston
        kappa, theta, sigma, rho, v0 = params
        temp1 = kappa - 1j * rho * sigma * u
        g = np.sqrt(sigma**2 * (u**2 + 1j * u) + temp1**2)
        power_term = 2 * kappa * theta / sigma**2
        log_numerator = np.log(np.cosh(g * T / 2) + (temp1 / g) * np.sinh(g * T / 2))
        log_denominator = (u**2 + 1j * u) * v0 / (g / np.tanh(g * T / 2) + temp1)
        phi = np.exp(1j * u * np.log(S0) + r * T * 1j * u - log_numerator - log_denominator)
    elif model == 'VG':  # Variance-Gamma
        sigma, nu, theta = params
        if nu == 0:
            mu = np.log(S0) + (r - q - theta - 0.5 * sigma**2) * T
            phi = np.exp(1j * u * mu) * np.exp((1j * theta * u - 0.5 * sigma**2 * u**2) * T)
        else:
            mu = np.log(S0) + (r - q + np.log(1 - theta * nu - 0.5 * sigma**2 * nu) / nu) * T
            phi = np.exp(1j * u * mu) * ((1 - 1j * nu * theta * u + 0.5 * nu * sigma**2 * u**2)**(-T / nu))
    return phi



In [5]:
# FFT Function to Calculate Option Prices
def calculate_fft_values(params, S0, K, r, q, T, alpha, eta, n, model):
    """
    Calculate option prices using FFT.

    Parameters:
        params (list): Model-specific parameters.
        S0 (float): Current stock price.
        K (float): Strike price.
        r (float): Risk-free interest rate.
        q (float): Dividend yield.
        T (float): Time to maturity.
        alpha (float): Damping factor.
        eta (float): Integration step size.
        n (int): FFT exponent (2^n points).
        model (str): Option pricing model.

    Returns:
        tuple: Log-strike values and option prices.
    """
    N = 2 ** n
    delta = (2 * np.pi / N) / eta
    beta = np.log(K)
    
    # Discount Factor
    discount_factor = np.exp(-r * T)
    
    # Range of Integration (nu_j)
    nuJ = np.arange(N) * eta
    psi_nuJ = compute_characteristic_function(nuJ - (alpha + 1) * 1j, params, S0, r, q, T, model) / (
        (alpha + 1j * nuJ) * (alpha + 1 + 1j * nuJ)
    )
    
    # FFT Computation
    km_values = beta + delta * np.arange(N)
    w_values = eta * np.ones(N)
    w_values[0] = eta / 2  # Adjusting the first weight
    x_values = np.exp(-1j * beta * nuJ) * discount_factor * psi_nuJ * w_values
    y_values = np.fft.fft(x_values)
    cT_km_values = (np.exp(-alpha * km_values) / np.pi) * np.real(y_values)
    
    return km_values, cT_km_values



In [6]:
# Pricing Options for Black-Scholes, Heston, and Variance-Gamma Models
def price_options_and_plot(params, S0, K, r, q, T, alpha, eta, n, model, title):
    km_values, cT_km_values = calculate_fft_values(params, S0, K, r, q, T, alpha, eta, n, model)
    
    # Convert Log-Strike to Normal Strike Prices
    strikes = np.exp(km_values)
    
    # Create a DataFrame for Results
    results = pd.DataFrame({'Strike Price': strikes, 'Option Price': cT_km_values})
    
    # Plotting the Results
    #plt.figure(figsize=(10, 6))
    #plt.plot(strikes, cT_km_values, label=f"{model} Option Prices", color='blue')
    #plt.axvline(x=K, color='red', linestyle='--', label='Strike Price (K)')
    #plt.title(f"{title} Option Prices using FFT")
    #plt.xlabel("Strike Price")
    #plt.ylabel("Option Price")
    #plt.legend()
    #plt.grid(True)
    #plt.show()
    
    return results



In [7]:
# Fetch Option Chain using yfinance
def fetch_option_chain(ticker, expiration_date):
    """
    Fetch option chain for a given ticker and expiration date.

    Parameters:
        ticker (str): Stock ticker symbol.
        expiration_date (str): Expiration date (YYYY-MM-DD).

    Returns:
        pd.DataFrame: Option chain data.
    """
    stock = yf.Ticker(ticker)
    option_chain = stock.option_chain(expiration_date)
    calls = option_chain.calls
    return calls

# Example: Fetch option chain for AAPL
expiration_date = "2025-01-03"  # Example expiration date
option_chain = fetch_option_chain(ticker, expiration_date)

# Display Option Chain
option_chain[['strike', 'lastPrice', 'impliedVolatility', 'volume']].head()

# Black-Scholes Parameters
model = 'BS'
volatility = stock_data['Daily Return'].std() * np.sqrt(252)  # Annualized volatility
params = [volatility]

print(f"Pricing options for {ticker} using Black-Scholes Model:")
bs_results = price_options_and_plot(params, S0, K, r, q, T, alpha, eta, n, model, "Black-Scholes")
print(bs_results.head())



Pricing options for AMZN using Black-Scholes Model:
   Strike Price  Option Price
0        231.04         13.72
1        232.46         13.19
2        233.89         12.67
3        235.33         12.17
4        236.78         11.68
